In [1]:
import numpy as np

In [2]:
zero = np.array([[1], [0]])
one = np.array([[0], [1]])

I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
H = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])

pauli = [I, X, Y, Z]

In [3]:
zero.shape

(2, 1)

In [4]:
def commutator(A, B):
    return A @ B - B @ A


def anti_commutator(A, B):
    return A @ B + B @ A


def expectation(rho, observable):
    return np.trace(rho @ observable)

In [5]:
# for a single qubit system
def pauli_decomposition(operator):
    N = len(pauli)
    a = np.zeros(shape=(N,), dtype=np.complex128)
    for i in range(N):
        a[i] = np.trace(operator @ pauli[i])
    return a / 2


# reconstruct the operator from the coefficients
def pauli_reconstruction(a):
    operator = np.zeros((2, 2)).astype(np.complex128)
    for i in range(len(pauli)):
        operator += a[i] * pauli[i]
    return operator

In [6]:
# column-wise
def vectorize(operator):
    return operator.flatten(order="F").reshape(-1, 1)


def devectorize(vector):
    N = np.sqrt(vector.size).astype(int)
    return vector.reshape(N, N, order="F")


np.random.seed(42)
A = np.random.randint(0, 10, (4, 4))
print(A)
print(vectorize(A).shape)
print(vectorize(A))
print(devectorize(vectorize(A)))

[[6 3 7 4]
 [6 9 2 6]
 [7 4 3 7]
 [7 2 5 4]]
(16, 1)
[[6]
 [6]
 [7]
 [7]
 [3]
 [9]
 [4]
 [2]
 [7]
 [2]
 [3]
 [5]
 [4]
 [6]
 [7]
 [4]]
[[6 3 7 4]
 [6 9 2 6]
 [7 4 3 7]
 [7 2 5 4]]


In [7]:
super_I = vectorize(I)
super_X = vectorize(X)
super_Y = vectorize(Y)
super_Z = vectorize(Z)

super_pauli = [super_I, super_X, super_Y, super_Z]

In [8]:
def pauli_vector_decomposition(vector):
    a = np.zeros((4, 1), dtype=np.complex128)
    for i in range(4):
        a[i] = super_pauli[i].conj().T @ vector
    return a / 2


def pauli_vector_reconstruction(a):
    vector = np.zeros((4, 1)).astype(np.complex128)
    for i in range(4):
        vector += a[i] * super_pauli[i]
    return vector

In [9]:
rho = H @ zero @ zero.T @ H.conj().T
super_rho = vectorize(rho)
c = pauli_vector_decomposition(super_rho)
print(c)
rho_reconstructed = pauli_vector_reconstruction(c)
print(rho_reconstructed)

[[0.5+0.j]
 [0.5+0.j]
 [0. +0.j]
 [0. +0.j]]
[[0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]]


In [10]:
# kraus operators for rotation operators is the same as the rotation operator itself


def x_rotation_operator(theta):
    return np.cos(theta / 2) * I - 1j * np.sin(theta / 2) * X


def y_rotation_operator(theta):
    return np.cos(theta / 2) * I - 1j * np.sin(theta / 2) * Y


def z_rotation_operator(theta):
    return np.cos(theta / 2) * I - 1j * np.sin(theta / 2) * Z

In [11]:
def unitary_channel(rho, U):
    return U @ rho @ U.T.conj()


def x_rotation_channel(rho, theta):
    return x_rotation_operator(theta) @ rho @ x_rotation_operator(theta).T.conj()


def y_rotation_channel(rho, theta):
    return y_rotation_operator(theta) @ rho @ y_rotation_operator(theta).T.conj()


def z_rotation_channel(rho, theta):
    return z_rotation_operator(theta) @ rho @ z_rotation_operator(theta).T.conj()


def bit_flip_channel(rho, p: int):
    return (1 - p) * rho + p * X @ rho @ X


def dephasing_channel(rho, p: int):
    return (1 - p / 2) * rho + (p / 2) * Z @ rho @ Z


# depolarizing channel is a special case of the pauli channel
def depolarizing_channel(rho, p: int):
    return (1 - (3 * p / 4)) * rho + (p / 4) * (X @ rho @ X + Y @ rho @ Y + Z @ rho @ Z)


def pauli_channel(rho, p: list):
    result = np.zeros(rho.shape, dtype=np.complex128)
    N = len(p)
    for i in range(N):
        if p[i] == 0:
            continue
        result += p[i] * pauli[i] @ rho @ pauli[i].T.conj()
    return result


def dephasing_kraus(p):
    E0 = np.sqrt(1 - p / 2) * I
    E1 = np.sqrt(p / 2) * Z
    return E0, E1


def depolarizing_kraus(p):
    E0 = np.sqrt(1 - 3 * p / 4) * I
    E1 = np.sqrt(p / 4) * X
    E2 = np.sqrt(p / 4) * Y
    E3 = np.sqrt(p / 4) * Z
    return E0, E1, E2, E3


def pauli_kraus(p):
    N = len(p)
    E = [np.sqrt(1 - (sum(p[1:]))) * I]
    for i in range(1, N):
        E.append(np.sqrt(p[i]) * pauli[i])
    return E[0], E[1], E[2], E[3]

In [12]:
def pauli_fidelity1(sigma, p):
    return np.trace(sigma.T.conj() @ pauli_channel(sigma, p)).real / len(sigma)


def pauli_fidelity2(sigma, p):
    N = len(p)
    result = 0
    for j in range(N):
        if np.allclose(anti_commutator(pauli[j], sigma), np.zeros(sigma.shape)):
            result -= p[j]
        else:
            result += p[j]
    return result


def W():
    N = 4
    w = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            w[i, j] = (-1) ** np.allclose(
                anti_commutator(pauli[i], pauli[j]), np.zeros(pauli[i].shape)
            )
    return w


def fidelity(p):
    return np.round(W().T @ p, 2)


# pauli transfer matrix of pauli channel
def pauli_transfer_matrix(p):
    return np.diag(fidelity(p))

In [13]:
p = [0.9, 0.02, 0.05, 0.03]
print(f"pauli fidelity: \n{fidelity(p)}")
print(f"pauli transfer matrix: \n{pauli_transfer_matrix(p)}")

pauli fidelity: 
[1.   0.84 0.9  0.86]
pauli transfer matrix: 
[[1.   0.   0.   0.  ]
 [0.   0.84 0.   0.  ]
 [0.   0.   0.9  0.  ]
 [0.   0.   0.   0.86]]


### Chi Matrix (Process Matrix or X-Matrix)


#### A representation of a quantum map using the Pauli basis

In [14]:
def process_matrix(kraus_operators):
    # expanding Kraus operators in terms of Pauli matrices
    E = kraus_operators
    coeff = np.zeros((len(E), 4)).astype(np.complex128)
    for i in range(len(E)):
        coeff[i] = pauli_decomposition(E[i])
    R, C = coeff.shape
    X = np.zeros((C, C)).astype(np.complex128)
    for i in range(R):
        for j in range(C):
            for k in range(C):
                X[j, k] += coeff[i, j] * coeff[i, k].conj()
    return X

In [15]:
def process_channel(rho, kraus_operators):
    result = np.zeros(rho.shape, dtype=np.complex128)
    X_matrix = process_matrix(kraus_operators)
    for j in range(4):
        for k in range(4):
            xjk = X_matrix[j, k]
            result += xjk * pauli[j] @ rho @ pauli[k].T.conj()
    return result

In [16]:
p = 0.5
process_matrix(dephasing_kraus(p)).round(2)

array([[0.75+0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.25+0.j]])

In [17]:
process_matrix([z_rotation_operator(np.pi / 4)]).round(2)

array([[0.85+0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.35j],
       [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  ],
       [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  ],
       [0.  -0.35j, 0.  +0.j  , 0.  +0.j  , 0.15+0.j  ]])

In [18]:
# apply the process matrix to the density matrix
rho = H @ zero @ zero.T @ H.conj().T
process_channel(rho, [z_rotation_operator(np.pi / 4)]).round(2)

array([[0.5 +0.j  , 0.35-0.35j],
       [0.35+0.35j, 0.5 +0.j  ]])

### Choi Matrix

In [19]:
def choi_matrix(kraus_operators):
    E = kraus_operators
    N = len(E)
    result = np.zeros((4, 4)).astype(np.complex128)
    for i in range(N):
        super_operator = vectorize(E[i])
        result += super_operator @ super_operator.T.conj()
    return result / 2

In [20]:
choi_matrix([z_rotation_operator(np.pi / 4)]).round(2)

array([[0.5 +0.j  , 0.  +0.j  , 0.  +0.j  , 0.35-0.35j],
       [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  ],
       [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  ],
       [0.35+0.35j, 0.  +0.j  , 0.  +0.j  , 0.5 +0.j  ]])

In [21]:
choi_matrix(pauli_kraus([0.9, 0.02, 0.05, 0.03])).round(2)

array([[ 0.46+0.j,  0.  +0.j,  0.  +0.j,  0.43+0.j],
       [ 0.  +0.j,  0.03+0.j, -0.01+0.j,  0.  +0.j],
       [ 0.  +0.j, -0.01+0.j,  0.03+0.j,  0.  +0.j],
       [ 0.43+0.j,  0.  +0.j,  0.  +0.j,  0.46+0.j]])

### Superoperators

In [22]:
def superoperator(unitary):
    N = len(unitary)
    S = np.zeros((4, 4), dtype=np.complex128)
    for i in range(N):
        S += np.kron(unitary[i].conj(), unitary[i])
    return S

In [23]:
superoperator([x_rotation_operator(np.pi / 4)]).round(2)

array([[0.85+0.j  , 0.  -0.35j, 0.  +0.35j, 0.15+0.j  ],
       [0.  -0.35j, 0.85+0.j  , 0.15+0.j  , 0.  +0.35j],
       [0.  +0.35j, 0.15+0.j  , 0.85+0.j  , 0.  -0.35j],
       [0.15+0.j  , 0.  +0.35j, 0.  -0.35j, 0.85+0.j  ]])

In [24]:
superoperator(pauli_kraus([0.9, 0.02, 0.05, 0.03])).round(2).real

array([[ 0.93,  0.  ,  0.  ,  0.07],
       [ 0.  ,  0.87, -0.03,  0.  ],
       [ 0.  , -0.03,  0.87,  0.  ],
       [ 0.07,  0.  ,  0.  ,  0.93]])

### Pauli Transfer Matrix

#### A representation of a quantum map using the Pauli basis


In [25]:
# general pauli transfer matrix
def transfer_matrix(p=None, channel=None, unitary=None):
    M = np.zeros((4, 4)).astype(np.complex128)
    for i in range(4):
        for j in range(4):
            if unitary is not None:
                M[i, j] = np.trace(pauli[i] @ channel(pauli[j], unitary))
            else:
                M[i, j] = np.trace(pauli[i] @ channel(pauli[j], p))
    return M / 2

In [26]:
p = 0.1
transfer_matrix(p, bit_flip_channel).real

array([[1. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. ],
       [0. , 0. , 0.8, 0. ],
       [0. , 0. , 0. , 0.8]])

In [27]:
p = [0.9, 0.02, 0.05, 0.03]
transfer_matrix(p, pauli_channel).real

array([[1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.84, 0.  , 0.  ],
       [0.  , 0.  , 0.9 , 0.  ],
       [0.  , 0.  , 0.  , 0.86]])

In [28]:
transfer_matrix(channel=unitary_channel, unitary=X).real

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0., -1.,  0.],
       [ 0.,  0.,  0., -1.]])

In [29]:
theta = np.pi / 3
transfer_matrix(theta, x_rotation_channel).real.round(2)

array([[ 1.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.5 , -0.87],
       [ 0.  ,  0.  ,  0.87,  0.5 ]])

#### Applying pauli transfer matrix of pauli channel instead of apply pauli channel directly

In [30]:
state = H @ zero
rho = state @ state.T.conj()
p = [0.9, 0.02, 0.05, 0.03]
print(f"density matrix before applying the pauli channel: \n{rho}")

density matrix before applying the pauli channel: 
[[0.5 0.5]
 [0.5 0.5]]


In [31]:
rho_prime = pauli_channel(rho, p)
print(f"density matrix after applying the pauli channel: \n{rho_prime}")
a_prime = pauli_decomposition(rho_prime)
print(
    f"coefficients of the density matrix after applying the pauli channel: \n{a_prime.T}"
)

density matrix after applying the pauli channel: 
[[0.5 +0.j 0.42+0.j]
 [0.42+0.j 0.5 +0.j]]
coefficients of the density matrix after applying the pauli channel: 
[0.5 +0.j 0.42+0.j 0.  +0.j 0.  +0.j]


In [32]:
a = pauli_decomposition(rho)
PTM = pauli_transfer_matrix(p)
print(f"pauli transfer matrix of pauli channel: \n{PTM.real}")
a_prime = PTM @ a
print(
    f"coefficients of the density matrix after applying the pauli transfer matrix of pauli channel: \n{a_prime.T}"
)
rho_prime = pauli_reconstruction(a_prime)
print(
    f"density matrix after apply the pauli transfer matrix of pauli channel: \n{rho_prime}"
)

pauli transfer matrix of pauli channel: 
[[1.   0.   0.   0.  ]
 [0.   0.84 0.   0.  ]
 [0.   0.   0.9  0.  ]
 [0.   0.   0.   0.86]]
coefficients of the density matrix after applying the pauli transfer matrix of pauli channel: 
[0.5 +0.j 0.42+0.j 0.  +0.j 0.  +0.j]
density matrix after apply the pauli transfer matrix of pauli channel: 
[[0.5 +0.j 0.42+0.j]
 [0.42+0.j 0.5 +0.j]]


#### Applying pauli transfer matrix of a certain channel on vectorized density matrix

In [33]:
state = H @ zero
rho = state @ state.T.conj()
U = y_rotation_operator(np.pi / 4)
print(f"density matrix before applying the y rotation channel: \n{rho}")

density matrix before applying the y rotation channel: 
[[0.5 0.5]
 [0.5 0.5]]


In [34]:
unitary_channel(rho, U)

array([[0.14644661+0.j, 0.35355339+0.j],
       [0.35355339+0.j, 0.85355339+0.j]])

In [35]:
super_rho = vectorize(rho)
S = superoperator([U])
super_rho_prime = S @ super_rho
rho_prime = devectorize(super_rho_prime)
print(f"density matrix after applying the unitary channel: \n{rho_prime}")

density matrix after applying the unitary channel: 
[[0.14644661+0.j 0.35355339+0.j]
 [0.35355339+0.j 0.85355339+0.j]]


### Pauli Twirling

In [36]:
def pauli_twirling(rho, p, channel: callable):
    result = np.zeros(rho.shape, dtype=np.complex128)
    for i in range(4):
        result += pauli[i] @ channel(pauli[i] @ rho @ pauli[i], p) @ pauli[i].T.conj()
    return result / 4

In [37]:
state = x_rotation_operator(np.pi / 4) @ zero
rho = state @ state.T.conj()
print(f"state before applying the pauli twirling: \n{state}")
print(f"density matrix before applying the pauli twirling: \n{rho.round(2)}")

state before applying the pauli twirling: 
[[0.92387953+0.j        ]
 [0.        -0.38268343j]]
density matrix before applying the pauli twirling: 
[[0.85+0.j   0.  +0.35j]
 [0.  -0.35j 0.15+0.j  ]]


In [38]:
state = x_rotation_operator(np.pi / 4) @ zero
rho = state @ state.T.conj()
theta = np.pi / 5
channel = x_rotation_channel
print(f"expectation: {expectation(pauli_twirling(rho, theta, channel), Z).round(2)}")

expectation: (0.57+0j)


##### X rotation channel

In [39]:
state = x_rotation_operator(np.pi / 4) @ zero
rho = state @ state.T.conj()
print(f"initial density matrix: \n{rho.round(2)}")
result = np.zeros((2, 2), dtype=np.complex128)
for i in range(4):
    curr = unitary_channel(rho, pauli[i])
    curr = unitary_channel(curr, x_rotation_operator(np.pi / 5))
    curr = unitary_channel(curr, pauli[i])
    result += curr / 4
print(f"final density matrix: \n{result.round(2)}")

initial density matrix: 
[[0.85+0.j   0.  +0.35j]
 [0.  -0.35j 0.15+0.j  ]]
final density matrix: 
[[0.79+0.j   0.  +0.29j]
 [0.  -0.29j 0.21+0.j  ]]


In [40]:
state = x_rotation_operator(np.pi / 4) @ zero
rho = state @ state.T.conj()
print(f"initial density matrix: \n{rho.round(2)}")
c = np.pi / 5
channel = x_rotation_channel

# apply the x rotation channel
print(f"density matrix after applying the channel: \n{channel(rho, c).round(2)}")

# apply the pauli twirling
print(
    f"density matrix after applying the pauli twirling: \n{pauli_twirling(rho, c, channel).round(2)}"
)

initial density matrix: 
[[0.85+0.j   0.  +0.35j]
 [0.  -0.35j 0.15+0.j  ]]
density matrix after applying the channel: 
[[0.58+0.j   0.  +0.49j]
 [0.  -0.49j 0.42+0.j  ]]
density matrix after applying the pauli twirling: 
[[0.79+0.j   0.  +0.29j]
 [0.  -0.29j 0.21+0.j  ]]


#### Chi matrix of pauli twirling

In [41]:
def twirling_chi(kraus_operators):
    return np.diag(process_matrix(kraus_operators).diagonal())


twirling_chi([x_rotation_operator(np.pi / 4)]).round(2)

array([[0.85+0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.15+0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j]])

#### pauli transfer matrix of pauli twirling

In [42]:
I_PTM = transfer_matrix(channel=unitary_channel, unitary=I).real
X_PTM = transfer_matrix(channel=unitary_channel, unitary=X).real
Y_PTM = transfer_matrix(channel=unitary_channel, unitary=Y).real
Z_PTM = transfer_matrix(channel=unitary_channel, unitary=Z).real

# generate a random matrix 4x4 with positive integer values
np.random.seed(42)
A = np.random.randint(0, 10, (4, 4))

print(f"A: {A}")
print(f"I: {I_PTM @ A @ I_PTM}")
print(f"X: {X_PTM @ A @ X_PTM}")
print(f"Y: {Y_PTM @ A @ Y_PTM}")
print(f"Z: {Z_PTM @ A @ Z_PTM}")

result = np.zeros((4, 4)).astype(np.complex128)
for i in range(4):
    result += (
        transfer_matrix(channel=unitary_channel, unitary=pauli[i])
        @ A
        @ transfer_matrix(channel=unitary_channel, unitary=pauli[i])
    )
print(f"result: \n{result.real / 4}")

A: [[6 3 7 4]
 [6 9 2 6]
 [7 4 3 7]
 [7 2 5 4]]
I: [[6. 3. 7. 4.]
 [6. 9. 2. 6.]
 [7. 4. 3. 7.]
 [7. 2. 5. 4.]]
X: [[ 6.  3. -7. -4.]
 [ 6.  9. -2. -6.]
 [-7. -4.  3.  7.]
 [-7. -2.  5.  4.]]
Y: [[ 6. -3.  7. -4.]
 [-6.  9. -2.  6.]
 [ 7. -4.  3. -7.]
 [-7.  2. -5.  4.]]
Z: [[ 6. -3. -7.  4.]
 [-6.  9.  2. -6.]
 [-7.  4.  3. -7.]
 [ 7. -2. -5.  4.]]
result: 
[[6. 0. 0. 0.]
 [0. 9. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 4.]]


In [43]:
def test(constant, channel):
    result = np.zeros((4, 4)).astype(np.complex128)
    for i in range(4):
        u = transfer_matrix(channel=unitary_channel, unitary=pauli[i])
        result += u @ transfer_matrix(constant, channel) @ u.T.conj()
    return result / 4

In [44]:
def twirling_PTM(constant, channel):
    return np.diag(transfer_matrix(constant, channel).diagonal())


# constant = 0.2
# channel = bit_flip_channel
constant = np.pi / 3
channel = x_rotation_channel
twirling_PTM(constant, channel).round(2).real

array([[1. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. ],
       [0. , 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0.5]])

In [45]:
state = x_rotation_operator(np.pi / 4) @ zero
rho = state @ state.T.conj()
print(f"density matrix before applying the twirling PTM: \n{rho}")
channel = x_rotation_channel
theta = np.pi / 3
print(
    f"density matrix after applying the pauli twirling: \n{pauli_twirling(rho, theta, channel).round(2)}"
)
a = pauli_decomposition(rho)
PTM = twirling_PTM(theta, channel)
a_prime = PTM @ a
rho_prime = pauli_reconstruction(a_prime).round(2)
print(f"density matrix after applying the twirling PTM: \n{rho_prime}")

density matrix before applying the twirling PTM: 
[[0.85355339+0.j         0.        +0.35355339j]
 [0.        -0.35355339j 0.14644661+0.j        ]]
density matrix after applying the pauli twirling: 
[[0.68+0.j   0.  +0.18j]
 [0.  -0.18j 0.32+0.j  ]]
density matrix after applying the twirling PTM: 
[[0.68+0.j   0.  +0.18j]
 [0.  -0.18j 0.32+0.j  ]]
